In [ ]:
%load_ext autoreload
%autoreload 2

import os
from dotenv import load_dotenv
load_dotenv()

import sys
sys.path.append('./../common')

## AML Related variables
runtime_in_cloud = os.environ.get("RUNTIME_IN_CLOUD")
run_tag = os.environ.get("RUN_NAME_TAG")
## Local variables

In [ ]:
### Initialize Variables
cloud_run = True #set to True if running in cloud

data_id="../keyword_correctness/data/experiment/sample_input_5records.jsonl"
tags={"data_id": data_id}

In [ ]:
# Initialize PF Client Utils

from pf_sdk_utils import PromptFlowUtils
import sys

pf_utils = PromptFlowUtils()
pf_utils.initialize_pf_client(cloud = cloud_run)

In [ ]:
def download_or_display_outputs(executed_runs, ran_in_cloud):
    """
    Downloads or displays the outputs of executed runs based on the value of `ran_in_cloud`.

    Args:
        executed_runs (list): A list of executed runs.
        ran_in_cloud (bool): A flag indicating whether the runs were executed in the cloud.

    Returns:
        None
    """
    if(ran_in_cloud):
        from output_utils import download_output
        output_runs_dir = "./runs"
        for run in executed_runs:
            if isinstance(run, str):
                output_file_path = f"{output_runs_dir}/{run}"
            else:
                output_file_path = f"{output_runs_dir}/{run.name}"
            download_output(run,output_file_path)
    else:
        pf_utils.display_local_runs(executed_runs)

In [ ]:
# Import the flows which are required to be executed in the notebook
# Initialize timestamp, which will server as unique id for all the runs performed in the notebook

from entities import Run
from flow_definitions import experiment_Step_1_flow, experiment_Step_2_flow, evaluation_flow
from datetime import datetime

timestamp = datetime.now().strftime('%d%H%M%S') 



In [ ]:
# This code execute the step 1 of experiment. Please select the variants you want to run
variants = experiment_Step_1_flow.variants
current_run_4_1 = Run(flow = experiment_Step_1_flow, tags=tags, 
                  runtime=runtime_in_cloud, data_id=data_id, 
                  linked_runs=[],
                  variants=variants, run_suffix=timestamp, env_vars = {"PF_WORKER_COUNT": "5"})

executed_runs_4_1 = pf_utils.execute(current_run = current_run_4_1)

allPassed = pf_utils.wait_for_run_completion(executed_runs_4_1)
if not allPassed:
    sys.exit(1)

In [ ]:
# This code execute the step 2 of experiment. Please select the variants you want to run and the linked runs from step 1
run_names_4_1 = [run.name for run in executed_runs_4_1]
variants = experiment_Step_2_flow.variants
current_run_4_2 = Run(flow = experiment_Step_2_flow, tags=tags, 
                  runtime=runtime_in_cloud,
                  linked_runs=run_names_4_1,
                  variants=variants, run_suffix=timestamp,env_vars = {"PF_WORKER_COUNT": "5"})

executed_runs_4_2 = pf_utils.execute(current_run = current_run_4_2)

allPassed = pf_utils.wait_for_run_completion(executed_runs_4_2)
if not allPassed:
    sys.exit(1)

In [ ]:
# This code execute the evaluation flow. Please select the linked runs from step 2
run_names_4_2 = [run.name for run in executed_runs_4_2]
eval_run = Run(flow = evaluation_flow, tags=tags, 
                  runtime=runtime_in_cloud,
                  linked_runs=run_names_4_2,
                  run_suffix=timestamp,env_vars = {"PF_WORKER_COUNT": "5"})

eval_runs = pf_utils.execute(current_run = eval_run)
allPassed = pf_utils.wait_for_run_completion(eval_runs)
if not allPassed:
    sys.exit(1)

In [ ]:
# Download or display the outputs of the executed runs
download_or_display_outputs(executed_runs_4_1, cloud_run)
download_or_display_outputs(executed_runs_4_2, cloud_run)
download_or_display_outputs(eval_runs, cloud_run)